In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tf.config.list_physical_devices("gpu")

[]

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16295790823125481960
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16037936578631759083
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [5]:
!nvidia-smi

Sun Apr 10 15:37:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    74W / 149W |    145MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
url = "/content/drive/MyDrive/Colab Notebooks/Dataset"

In [7]:
IMAGE_SIZE = 255
BATCH_SIZE = 32
EPOCHS = 50
CHANNELS = 3

In [8]:
dataset =  keras.preprocessing.image_dataset_from_directory(url, image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 1578 files belonging to 3 classes.


In [9]:
labels = dataset.class_names

In [10]:
labels

['benign', 'malignant', 'normal']

In [11]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 255, 255, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
for i, j in dataset.take(1):
  print(i.numpy())
  print(j.numpy())

[[[[  1.1658592    1.1658592    1.1658592 ]
   [  2.5051897    2.5051897    2.5051897 ]
   [204.75201    204.75201    204.75201   ]
   ...
   [123.276794   123.276794   123.276794  ]
   [  1.5948181    1.5948181    1.5948181 ]
   [  1.6794577    1.6794577    1.6794577 ]]

  [[202.43842    202.43842    202.43842   ]
   [ 17.9671      17.9671      17.9671    ]
   [200.81314    200.81314    200.81314   ]
   ...
   [122.397354   122.397354   122.397354  ]
   [ 15.057253    15.057253    15.057253  ]
   [  2.615507     2.615507     2.615507  ]]

  [[168.46666    168.46666    168.46666   ]
   [ 18.889587    18.889587    18.889587  ]
   [203.71857    203.71857    203.71857   ]
   ...
   [120.54151    120.54151    120.54151   ]
   [ 94.67401     94.67401     94.67401   ]
   [  0.42435953   0.42435953   0.42435953]]

  ...

  [[ 16.735321    16.735321    16.735321  ]
   [ 17.814205    17.814205    17.814205  ]
   [ 18.911774    18.911774    18.911774  ]
   ...
   [ 29.347214    29.347214    29.3

In [12]:
rescaling = keras.Sequential([
                              keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [13]:
model = keras.Sequential([
                          rescaling,
                          keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
                          keras.layers.MaxPooling2D((2, 2)),
                          keras.layers.Conv2D(64, (3, 3), activation="relu"),
                          keras.layers.MaxPooling2D((2, 2)),
                          keras.layers.Conv2D(128, (3, 3), activation="relu"),
                          keras.layers.MaxPooling2D((2, 2)),
                          keras.layers.Conv2D(128, (3, 3), activation="relu"),
                          keras.layers.MaxPooling2D((2, 2)),
                          keras.layers.Conv2D(128, (3, 3), activation="relu"),
                          keras.layers.MaxPooling2D((2, 2)),
                          keras.layers.Conv2D(128, (3, 3), activation="relu"),
                          keras.layers.MaxPooling2D((2, 2)),
                          keras.layers.Flatten(),
                          keras.layers.Dense(256, activation="relu"),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(128, activation="relu"),
                          keras.layers.Dropout(0.1),
                          keras.layers.Dense(3, activation="softmax")
])

In [14]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [15]:
train_df = dataset.take(int(len(dataset)*0.8))
test_df = dataset.skip(int(len(dataset)*0.8))

In [16]:
len(test_df)

10

In [17]:
val_df = test_df.take(5)
test_df = test_df.skip(5)

In [18]:
model.fit(train_df, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=val_df)

Epoch 1/50
40/40 [==============================] - 218s 4s/step - loss: 0.8986 - accuracy: 0.5570 - val_loss: 0.8502 - val_accuracy: 0.5188
Epoch 2/50
40/40 [==============================] - 19s 432ms/step - loss: 0.8090 - accuracy: 0.5859 - val_loss: 0.7626 - val_accuracy: 0.6438
Epoch 3/50
40/40 [==============================] - 18s 430ms/step - loss: 0.7315 - accuracy: 0.6914 - val_loss: 0.6309 - val_accuracy: 0.7000
Epoch 4/50
40/40 [==============================] - 18s 427ms/step - loss: 0.6448 - accuracy: 0.7156 - val_loss: 0.6002 - val_accuracy: 0.7375
Epoch 5/50
40/40 [==============================] - 18s 430ms/step - loss: 0.5604 - accuracy: 0.7547 - val_loss: 0.5576 - val_accuracy: 0.7063
Epoch 6/50
40/40 [==============================] - 18s 422ms/step - loss: 0.5052 - accuracy: 0.7672 - val_loss: 0.4841 - val_accuracy: 0.8062
Epoch 7/50
40/40 [==============================] - 18s 419ms/step - loss: 0.4522 - accuracy: 0.7828 - val_loss: 0.5049 - val_accuracy: 0.7937
E

In [19]:
model.evaluate(test_df)

5/5 [==============================] - 9s 139ms/step - loss: 0.2937 - accuracy: 0.9493


[0.2936643064022064, 0.9492753744125366]

In [20]:
model.save("lungsDiseaseModel.h5")